In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Post-training Quantization

In [39]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np

# data
data = pd.read_csv("sample_google_scholar.csv")
data = data.dropna()

def convert_first_ten_characters_into_tensor(data):
  first_ten_characters = data[:10]
  converted = [ord(char)/256 for char in first_ten_characters]
  while(len(converted)<10):
    converted.append(0.0)
  return np.array(converted)

converted_affiliation = data['affiliation'].map(convert_first_ten_characters_into_tensor)
affiliation = np.vstack(converted_affiliation.values)
converted_email = data['email'].str.contains('.edu')
labels = converted_email.values

# model
input_shape = 10
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(128, activation='relu', name='layer1'),
        layers.Dense(64, activation='relu', name='layer2'),
        layers.Dense(1, activation='sigmoid', name='layer3'),
    ]
)

loss = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()
model.compile(loss=loss, optimizer=optimizer)

# model fit
model.fit(affiliation, labels, batch_size=16, epochs=5, validation_split=0.2)

Epoch 1/5
3/3 [==============================] - 1s 76ms/step - loss: 0.6962 - val_loss: 0.7078
Epoch 2/5
3/3 [==============================] - 0s 11ms/step - loss: 0.6906 - val_loss: 0.7131
Epoch 3/5
3/3 [==============================] - 0s 11ms/step - loss: 0.6895 - val_loss: 0.7096
Epoch 4/5
3/3 [==============================] - 0s 10ms/step - loss: 0.6868 - val_loss: 0.7186
Epoch 5/5
3/3 [==============================] - 0s 10ms/step - loss: 0.6865 - val_loss: 0.7137


In [40]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 128)               1408      
                                                                 
 layer2 (Dense)              (None, 64)                8256      
                                                                 
 layer3 (Dense)              (None, 1)                 65        
                                                                 
Total params: 9729 (38.00 KB)
Trainable params: 9729 (38.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [43]:
tensorflow_model_path = "./tf_model"
model.save(tensorflow_model_path)

In [44]:
model.save('model.hdf5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [45]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

In [46]:
tfl_model = converter.convert()

In [47]:
open("tfl_model.tflite", "wb").write(tfl_model)

22444

In [48]:
import os
print("Original model in Mb: ", os.path.getsize('model.hdf5')/float(2**20))

Original model in Mb:  0.14165496826171875


In [49]:
print("Quantized model in Mb: ", os.path.getsize('tfl_model.tflite')/ float(2**20))

Quantized model in Mb:  0.021404266357421875


## Post- training quantization - Full integer quantization

In [50]:
def gen_rep():
  data = affiliation.astype(np.float32)
  data = tf.data.Dataset.from_tensor_slices(data).batch(1)
  for i in data.take(BATCH_SIZE):
    yield [i]

In [54]:
BATCH_SIZE=16
converter = tf.lite.TFLiteConverter.from_saved_model('./tf_model/')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = gen_rep
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type=tf.int8 # tf.uint8
converter.inference_output_type=tf.int8 # tf.uint8

In [55]:
tflite_quant_model = converter.convert()

In [56]:
open("tflite_quant_model.tflite", "wb").write(tflite_quant_model)

12776

In [57]:
print("Original model in Mb:", os.path.getsize('tfl_model.tflite')/float(2**20))

Original model in Mb: 0.021404266357421875


In [58]:
print("Quantized model (Full integer quantization) in Mb:", os.path.getsize('tflite_quant_model.tflite')/float(2**20))

Quantized model (Full integer quantization) in Mb: 0.01218414306640625


## Performing quantization aware training

In [62]:
import tensorflow_model_optimization as tfmot

In [63]:
# model
input_shape = 10
model = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Dense(128, activation='relu', name='layer1'),
    layers.Dense(64, activation='relu', name='layer2'),
    layers.Dense(1, activation='sigmoid', name='layer3'),
])

In [64]:
q_aware_model = tfmot.quantization.keras.quantize_model(model)

In [70]:
q_aware_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
q_aware_model.fit(affiliation, labels, batch_size=16, epochs=5, validation_split=0.2)

In [72]:
q_aware_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLa  (None, 10)                3         
 yer)                                                            
                                                                 
 quant_layer1 (QuantizeWrap  (None, 128)               1413      
 perV2)                                                          
                                                                 
 quant_layer2 (QuantizeWrap  (None, 64)                8261      
 perV2)                                                          
                                                                 
 quant_layer3 (QuantizeWrap  (None, 1)                 70        
 perV2)                                                          
                                                                 
Total params: 9747 (38.07 KB)
Trainable params: 9729 (